In [1]:
import sys, time, random, math, cv2, os
import numpy as np

sys.path.insert(0, '../src')
from robot import Robot

In [2]:
dimensions = (48,3)

near = 0.7

path = "Q_matrix.txt"
on_disk = path in os.listdir()

if on_disk:
    f = open(path, "r")
    
    t = f.read().split()
    random_rate = float(t[0])
    learning_rate = float(t[1])
    
    Q = np.reshape(np.array([float(i) for i in t[2:]]), dimensions)

    f.close()
else:
    random_rate = 0.9
    learning_rate = 0.6
    Q = np.zeros(dimensions)


In [3]:
def cicle(ang):
    return (ang + 2 * math.pi) % (2*math.pi)

def get_distance_from_goal(a,b):
    return np.linalg.norm(a-b)

def get_angular_distance(v1,v2):

    v1_aux = v1 / np.linalg.norm(v1)
    v2_aux = v2 / np.linalg.norm(v2)

    theta = np.arccos(np.clip(np.dot(v1_aux, v2_aux), -1.0, 1.0))
    rot_theta = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    theta = theta*180.0/math.pi
    theta = (360.0+theta)%360.0

    if abs(np.dot(np.dot(rot_theta,v2_aux),v1_aux)-1.0) > 1e-2:
        theta = 360.0 - theta

    return theta

def get_state(us_distances, current_position, current_angle, goal):
    angle_distance = get_angular_distance(goal - current_position, np.array([math.cos(current_angle),math.sin(current_angle)]))
    if angle_distance <= 5 or angle_distance >= 355: a = 1
    elif angle_distance > 180: a = 2
    else: a = 0
        
    b = 0
    b += 8 * (us_distances[2] < near)
    b += 4 * (us_distances[3] < near)
    b += 2 * (us_distances[4] < near)
    b += 1 * (us_distances[5] < near)
    
    return 16 * a + b

def get_action(state, random_rate):
    global Q, actions
    r = random.random() < random_rate

    if r:
        i = random.randint(0, len(actions) - 1)
    else:
        maximum = 0
        i = 0
        for j in range(len(Q[state])):
            if Q[state][j] > maximum:
                maximum = Q[state][j]
                i = j

    return i

def get_reward(us_distances, current_position, current_angle, goal):
    distance = get_distance_from_goal(current_position, goal)
    if distance <= 0.3:
        return 1000
    
    minimum = 10
    for i in us_distances[2:6]:
        minimum = min(minimum, i)
    if minimum <= 0.3:
        return -1000
    
    angle_distance = get_angular_distance(goal - current_position, np.array([math.cos(current_angle),math.sin(current_angle)]))
    if angle_distance > 180: angle_distance = 360 - angle_distance
    
    return 1 * (180 - angle_distance) + 1 * (100 - distance)
    

In [4]:
robot = Robot("#0")
actions = [(-5, 5), (10, 10), (5, -5)]

max_iterations = 10
max_steps = 100
iterations = 0
goal = np.array([13,0])
print(Q)
while iterations < max_iterations:
    step = 0

    us_distances = robot.read_ultrassonic_sensors()

    current_position = np.array(robot.get_current_position()[:-1])
    current_angle = cicle(robot.get_current_orientation()[2])
    
    state = get_state(us_distances, current_position, current_angle, goal)

    while robot.get_connection_status() != -1 and step < max_steps:
        print(step)
        i = get_action(state, random_rate)
        
        robot.set_left_velocity(actions[i][0])
        robot.set_right_velocity(actions[i][1])
        time.sleep(0.1)
        
        us_distances = robot.read_ultrassonic_sensors()

        current_position = np.array(robot.get_current_position()[:-1])
        current_angle = robot.get_current_orientation()[2]

        reward = get_reward(us_distances, current_position, current_angle, goal)
        
        best_q = np.amax(Q[state])
        
        Q[state][i] = learning_rate * (reward + 0.99 * best_q - Q[state][i])
        
        if reward == 1000:
            goal = -goal
            break
        
        if reward == -1000:
            robot.set_left_velocity(-5)
            robot.set_right_velocity(-5)
            time.sleep(1)
            robot.set_left_velocity(0)
            robot.set_right_velocity(0)

        state = get_state(us_distances, current_position, current_angle, goal)
        
        step += 1
    
    random_rate = max(0.1, random_rate * 0.95)
    learning_rate = max(0.1, learning_rate * 0.99)
    
    f = open(path, "w")
    f.write(str(random_rate) + "\n")
    f.write(str(learning_rate) + "\n")
    
    for i in Q:
        for j in i:
            f.write(str(j) + " ")
        f.write("\n")
        
    f.close()

    iterations += 1
    
    

print(Q)

Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1#0 connected.
 Pioneer_p3dx_ultrasonicSensor2#0 connected.
 Pioneer_p3dx_ultrasonicSensor3#0 connected.
 Pioneer_p3dx_ultrasonicSensor4#0 connected.
 Pioneer_p3dx_ultrasonicSensor5#0 connected.
 Pioneer_p3dx_ultrasonicSensor6#0 connected.
 Pioneer_p3dx_ultrasonicSensor7#0 connected.
 Pioneer_p3dx_ultrasonicSensor8#0 connected.
 Pioneer_p3dx_ultrasonicSensor9#0 connected.
 Pioneer_p3dx_ultrasonicSensor10#0 connected.
 Pioneer_p3dx_ultrasonicSensor11#0 connected.
 Pioneer_p3dx_ultrasonicSensor12#0 connected.
 Pioneer_p3dx_ultrasonicSensor13#0 connected.
 Pioneer_p3dx_ultrasonicSensor14#0 connected.
 Pioneer_p3dx_ultrasonicSensor15#0 connected.
 Pioneer_p3dx_ultrasonicSensor16#0 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.
[[ 134.37872852  122.38370528  141.11126936]
 [ 127.13067097   86.45726611  108.15140488]
 [   0.            0.           63.68194027

KeyboardInterrupt: 